In [53]:
import sys
import os
import pandas as pd

# Add the project root to the Python path
sys.path.append(os.path.join(os.getcwd(), '..'))

# Import the project modules
from src.config import setup_logging
from src.extract import fetch_statsbomb_event_data
from src.transform import transform_to_shot_events

# Reload modules when code is changed (uncomment for development)
%load_ext autoreload
%autoreload 2

# Init logging
logger = setup_logging(log_file="../logs/chance_creation.log")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [54]:
events = fetch_statsbomb_event_data()
list(events.columns)

2025-10-04 00:33:45,989 - src.extract.statsbomb_data - INFO - Fetching StatsBomb event data for Europe - UEFA Euro - 2024 - male
/opt/anaconda3/lib/python3.12/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/statsbombpy/api_client.py:21: No

['50_50',
 'bad_behaviour_card',
 'ball_receipt_outcome',
 'ball_recovery_offensive',
 'ball_recovery_recovery_failure',
 'block_deflection',
 'block_offensive',
 'block_save_block',
 'carry_end_location',
 'clearance_aerial_won',
 'clearance_body_part',
 'clearance_head',
 'clearance_left_foot',
 'clearance_other',
 'clearance_right_foot',
 'counterpress',
 'dribble_no_touch',
 'dribble_nutmeg',
 'dribble_outcome',
 'dribble_overrun',
 'duel_outcome',
 'duel_type',
 'duration',
 'foul_committed_advantage',
 'foul_committed_card',
 'foul_committed_offensive',
 'foul_committed_penalty',
 'foul_committed_type',
 'foul_won_advantage',
 'foul_won_defensive',
 'foul_won_penalty',
 'goalkeeper_body_part',
 'goalkeeper_end_location',
 'goalkeeper_outcome',
 'goalkeeper_penalty_saved_to_post',
 'goalkeeper_position',
 'goalkeeper_punched_out',
 'goalkeeper_shot_saved_off_target',
 'goalkeeper_shot_saved_to_post',
 'goalkeeper_success_in_play',
 'goalkeeper_technique',
 'goalkeeper_type',
 'id'

In [55]:
shots = transform_to_shot_events(events)
shots.info()

2025-10-04 00:33:58,338 - src.transform.shot_events - INFO - Transforming 187858 records from events data to shot events.
2025-10-04 00:34:00,213 - src.transform.shot_events - INFO - Transformed 1340 records from events data to shot events.
2025-10-04 00:34:00,217 - src.transform.shot_events - INFO - Shots from set piece: 696
2025-10-04 00:34:00,221 - src.transform.shot_events - INFO - Shots from open play: 644


<class 'pandas.core.frame.DataFrame'>
Index: 1340 entries, 184643 to 185982
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   match_id               1340 non-null   int64  
 1   team                   1340 non-null   object 
 2   player                 1340 non-null   object 
 3   location               1340 non-null   object 
 4   timestamp              1340 non-null   object 
 5   possession             1340 non-null   int64  
 6   type                   1340 non-null   object 
 7   shot_from_set_piece    1340 non-null   bool   
 8   shot_type              1340 non-null   object 
 9   shot_aerial_won        133 non-null    object 
 10  shot_body_part         1340 non-null   object 
 11  shot_end_location      1340 non-null   object 
 12  shot_first_time        380 non-null    object 
 13  shot_follows_dribble   1 non-null      object 
 14  shot_one_on_one        53 non-null     object 
 15  sh

In [56]:
shots.head()

,match_id,team,player,location,timestamp,possession,type,shot_from_set_piece,shot_type,shot_aerial_won,...,shot_end_location,shot_first_time,shot_follows_dribble,shot_one_on_one,shot_outcome,shot_redirect,shot_saved_off_target,shot_saved_to_post,shot_statsbomb_xg,shot_technique
184643,3942819,Netherlands,Xavi Simons,"[99.6, 51.2]",00:06:41.445,11,Shot,False,Open Play,NaN,...,"[120.0, 37.4, 2.0]",NaN,NaN,NaN,Goal,NaN,NaN,NaN,0.048935,Normal
184644,3942819,England,Harry Kane,"[88.2, 36.6]",00:12:39.772,17,Shot,False,Open Play,NaN,...,"[118.3, 37.6, 0.3]",NaN,NaN,NaN,Saved,NaN,NaN,NaN,0.028932,Normal
184645,3942819,England,Bukayo Saka,"[104.5, 41.6]",00:13:41.900,18,Shot,True,Open Play,NaN,...,"[106.2, 41.7]",NaN,NaN,NaN,Blocked,NaN,NaN,NaN,0.071750,Normal
184646,3942819,England,Harry Kane,"[108.1, 40.0]",00:17:34.626,19,Shot,False,Penalty,NaN,...,"[120.0, 36.5, 0.2]",NaN,NaN,NaN,Goal,NaN,NaN,NaN,0.783500,Normal
184647,3942819,England,Phil Foden,"[117.0, 47.6]",00:22:28.596,27,Shot,False,Open Play,NaN,...,"[117.7, 46.1, 0.3]",NaN,NaN,NaN,Saved,NaN,NaN,NaN,0.188995,Normal


In [26]:
general_cols = [
    "id", "match_id", "timestamp", "location", "possession", "play_pattern", "type", "team", "player", "position", "duration", "shot_statsbomb_xg", "pass_type"
]

In [ ]:
shot_cols = [
    "match_id", "team", "player", "location", "timestamp", "possession", "type", "shot_type", "shot_aerial_won", "shot_body_part", "shot_end_location", "shot_first_time", "shot_follows_dribble", 
    "shot_one_on_one", "shot_outcome", "shot_redirect", "shot_saved_off_target", "shot_saved_to_post", "shot_statsbomb_xg", "shot_technique"
]
events[events['shot_type'] == 'Corner'][shot_cols]

# Not relevant for this analysis: these are olympic goal attempts

,match_id,team,player,location,timestamp,possession,type,shot_type,shot_aerial_won,shot_body_part,shot_end_location,shot_first_time,shot_follows_dribble,shot_one_on_one,shot_outcome,shot_redirect,shot_saved_off_target,shot_saved_to_post,shot_statsbomb_xg,shot_technique
184839,3942227,Switzerland,Xherdan Shaqiri,"[119.3, 79.1]",00:11:40.450,156,Shot,Corner,NaN,Left Foot,"[120.0, 43.8, 2.8]",NaN,NaN,True,Post,NaN,NaN,NaN,0.00018,Normal
184912,3938645,Ukraine,Ruslan Malinovskiy,"[119.0, 79.3]",00:37:10.885,140,Shot,Corner,NaN,Left Foot,"[119.8, 43.2, 0.8]",NaN,NaN,True,Saved,NaN,NaN,NaN,0.00018,Normal
184985,3941021,Netherlands,Memphis Depay,"[119.2, 79.2]",00:42:02.239,61,Shot,Corner,NaN,Right Foot,"[120.0, 38.1, 4.5]",NaN,NaN,True,Off T,NaN,NaN,NaN,0.00018,Normal
185650,3938638,Romania,Nicolae Claudiu Stanciu,"[120.1, 0.8]",00:38:43.510,59,Shot,Corner,NaN,Right Foot,"[120.0, 38.6, 3.0]",NaN,NaN,True,Post,NaN,NaN,NaN,0.00018,Normal


In [11]:
events[(events["match_id"] == 3941021) & (events["possession"] == 61)][shot_cols]

,match_id,team,player,position,timestamp,possession,type,shot_type,shot_aerial_won,shot_body_part,shot_end_location,shot_first_time,shot_follows_dribble,shot_one_on_one,shot_outcome,shot_redirect,shot_saved_off_target,shot_saved_to_post,shot_statsbomb_xg,shot_technique
180610,3941021,Romania,Florin Constantin Niţă,Goalkeeper,00:42:03.720,61,Goal Keeper,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
184985,3941021,Netherlands,Memphis Depay,Center Forward,00:42:02.239,61,Shot,Corner,NaN,Right Foot,"[120.0, 38.1, 4.5]",NaN,NaN,True,Off T,NaN,NaN,NaN,0.00018,Normal


In [14]:
events["play_pattern"].value_counts()

play_pattern
Regular Play      76594
From Throw In     41870
From Free Kick    29799
From Goal Kick    16090
From Corner        7722
From Keeper        6776
From Kick Off      6238
From Counter       1514
Other              1255
Name: count, dtype: int64

In [21]:
events[(events["play_pattern"] == "From Throw In") & (events["type"] == "Shot")][general_cols]

,id,match_id,timestamp,location,possession,play_pattern,type,team,player,position,duration,shot_statsbomb_xg
184645,825afa97-dac6-4d4b-b16d-585361c9ff61,3942819,00:13:41.900,"[104.5, 41.6]",18,From Throw In,Shot,England,Bukayo Saka,Right Wing Back,0.083512,0.071750
184655,1edbf02d-991c-47af-bd53-e82f3485e9a6,3942819,00:31:57.558,"[110.4, 38.9]",94,From Throw In,Shot,Netherlands,Wout Weghorst,Center Forward,0.033592,0.057836
184656,ac5471c7-f47d-4553-83fd-aa8ad4152fdf,3942819,00:31:58.901,"[103.5, 45.0]",94,From Throw In,Shot,Netherlands,Xavi Simons,Right Wing,0.968812,0.062322
184662,7f58b69c-8311-493f-91dd-d2a57ac07387,3943043,00:27:27.748,"[101.2, 49.8]",44,From Throw In,Shot,Spain,Fabián Ruiz Peña,Left Defensive Midfield,0.178330,0.048117
184667,64e7e679-4246-4172-b59e-56905a09ffbf,3943043,00:01:09.977,"[109.9, 28.0]",81,From Throw In,Shot,Spain,Nicholas Williams Arthuer,Left Wing,0.544491,0.112482
...,...,...,...,...,...,...,...,...,...,...,...,...
185954,2649c196-4337-4ce5-bd66-9b2b7f067bf9,3930159,00:20:25.214,"[115.5, 42.1]",106,From Throw In,Shot,Hungary,Barnabás Varga,Right Attacking Midfield,0.328855,0.579687
185956,afd6867e-3222-4985-85fc-ecfca93746f9,3930159,00:25:39.234,"[108.7, 25.6]",112,From Throw In,Shot,Switzerland,Dan Ndoye,Right Attacking Midfield,0.207698,0.052711
185962,30cfa4e6-8610-4341-a904-5728fa572a3e,3930158,00:09:55.118,"[101.1, 42.0]",19,From Throw In,Shot,Germany,Florian Wirtz,Left Wing,1.185566,0.064226
185963,8b2965f3-ef80-48c0-adf1-6e08460c93b8,3930158,00:18:43.626,"[106.4, 35.4]",29,From Throw In,Shot,Germany,Jamal Musiala,Right Wing,0.542468,0.093475


In [27]:
events[(events["match_id"] == 3942819) & (events["possession"] == 18)][general_cols].sort_values(by="timestamp")

,id,match_id,timestamp,location,possession,play_pattern,type,team,player,position,duration,shot_statsbomb_xg,pass_type
458,17d8c5f4-4570-4ed2-927a-46050e141d68,3942819,00:12:49.440,"[106.2, 0.1]",18,From Throw In,Pass,England,Kieran Trippier,Left Wing Back,2.425623,NaN,Throw-in
54341,2460bad4-0b4b-4782-9634-d621a88c7d5c,3942819,00:12:51.865,"[83.2, 4.4]",18,From Throw In,Ball Receipt*,England,Marc Guehi,Left Center Back,NaN,NaN,NaN
459,ecae76af-66d5-4192-9038-f8910abb484e,3942819,00:12:53.772,"[80.5, 6.0]",18,From Throw In,Pass,England,Marc Guehi,Left Center Back,0.967488,NaN,NaN
105951,8ba9f729-58ae-4fcb-ac54-5a68b0ebcfdc,3942819,00:12:54.740,"[87.0, 19.3]",18,From Throw In,Carry,England,Kobbie Mainoo,Right Defensive Midfield,0.557970,NaN,NaN
54342,35acda18-194d-4b05-8bb2-428d43fa4786,3942819,00:12:54.740,"[87.0, 19.3]",18,From Throw In,Ball Receipt*,England,Kobbie Mainoo,Right Defensive Midfield,NaN,NaN,NaN
460,50bce31a-2a3b-49ac-90ef-4e62c43e6d96,3942819,00:12:55.298,"[85.6, 17.7]",18,From Throw In,Pass,England,Kobbie Mainoo,Right Defensive Midfield,1.245307,NaN,NaN
105952,da34850b-a3f7-4f20-8f55-a919213b6d46,3942819,00:12:56.543,"[75.0, 9.1]",18,From Throw In,Carry,England,Declan Rice,Left Defensive Midfield,0.853965,NaN,NaN
54343,89c3d892-d805-48bf-9458-acf52bd14e38,3942819,00:12:56.543,"[75.0, 9.1]",18,From Throw In,Ball Receipt*,England,Declan Rice,Left Defensive Midfield,NaN,NaN,NaN
461,56c0d9a1-7a8f-4ee5-9b3c-5c0c536538f2,3942819,00:12:57.397,"[74.1, 9.1]",18,From Throw In,Pass,England,Declan Rice,Left Defensive Midfield,1.721768,NaN,NaN
105953,18ae90f0-653f-44dd-b398-e218a03a2f0e,3942819,00:12:59.119,"[61.5, 34.9]",18,From Throw In,Carry,England,John Stones,Center Back,1.965617,NaN,NaN
